In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [36]:
query2=f"""
SELECT * FROM `perceptive-ivy-290216.f1_api.race_lap_time`
# WHERE YEAR=2024
# AND GP='Saudi Arabian Grand Prix'
"""
track3=pandas_gbq.read_gbq(query2,project_id,dialect='standard')

Downloading: 100%|██████████|


In [37]:
query_all_laps=f"""
SELECT * FROM `perceptive-ivy-290216.f1_api.race_lap_time`
UNION ALL
SELECT * FROM `perceptive-ivy-290216.f1_api.final_driver_positions_all`
"""
track_all_laps=pandas_gbq.read_gbq(query_all_laps,project_id,dialect='standard')

Downloading: 100%|██████████|


In [38]:
race="Dutch Grand Prix"
years=2024

In [39]:
track2=track3[(track3["GP"]==race)&(track3["Year"]==years)]

In [40]:
track_all_laps[track_all_laps["Driver"]=="BOT"].tail()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
466300,0 days 02:10:26.475000,BOT,77,0 days 00:01:11.356000,62.0,3.0,NaT,NaT,0 days 00:00:17.840000,0 days 00:00:32.015000,0 days 00:00:21.501000,0 days 02:09:32.955000,0 days 02:10:04.970000,0 days 02:10:26.471000,289.0,225.0,272.0,298.0,False,HARD,20.0,True,Kick Sauber,0 days 02:09:15.119000,NaT,1,17.0,False,,False,True,2024,Austrian Grand Prix
466301,0 days 02:21:50.231000,BOT,77,0 days 00:01:34.094000,55.0,2.0,NaT,NaT,0 days 00:00:26.921000,0 days 00:00:31.954000,0 days 00:00:35.219000,0 days 02:20:43.060000,0 days 02:21:15.014000,0 days 02:21:50.233000,205.0,248.0,248.0,267.0,False,MEDIUM,13.0,True,Kick Sauber,0 days 02:20:16.137000,NaT,4,13.0,False,,False,False,2024,Canadian Grand Prix
466302,0 days 02:34:33.100000,BOT,77,0 days 00:01:38,38.0,4.0,NaT,NaT,0 days 00:00:35.557000,0 days 00:00:43.663000,0 days 00:00:18.780000,0 days 02:33:30.556000,0 days 02:34:14.219000,0 days 02:34:32.999000,270.0,298.0,260.0,296.0,False,HARD,16.0,True,Kick Sauber,0 days 02:32:55.100000,NaT,1,12.0,False,,False,True,2024,Japanese Grand Prix
466303,0 days 01:19:44.777000,BOT,77,0 days 00:01:42.359000,14.0,2.0,NaT,NaT,0 days 00:00:27.546000,0 days 00:00:30.783000,0 days 00:00:44.030000,0 days 01:18:29.948000,0 days 01:19:00.731000,0 days 01:19:44.761000,263.0,259.0,254.0,304.0,False,HARD,5.0,True,Kick Sauber,0 days 01:18:02.418000,NaT,1,13.0,False,,False,True,2024,Chinese Grand Prix
466304,0 days 02:14:00.474000,BOT,77,0 days 00:01:20.369000,56.0,3.0,NaT,NaT,0 days 00:00:23.705000,0 days 00:00:32.280000,0 days 00:00:24.384000,0 days 02:13:03.780000,0 days 02:13:36.060000,0 days 02:14:00.444000,272.0,276.0,276.0,304.0,False,HARD,28.0,True,Kick Sauber,0 days 02:12:40.105000,NaT,1,15.0,False,,False,True,2024,Spanish Grand Prix


In [41]:
#Get Final Pos to order the final plot
track_all_laps_2=track_all_laps[(track_all_laps["GP"]==race)&(track_all_laps["Year"]==years)]
final_pos=track_all_laps_2[track_all_laps_2["LapNumber"]==track_all_laps_2.LapNumber.max()][["Driver","Position"]]

In [42]:
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]
Drivers=(track2['Driver'].unique())

In [43]:
stints = track2[["Driver", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "Stint", "Compound"]).count().reset_index()
stints = stints.rename(columns={"LapNumber": "StintLength"})
stints.head()

,Driver,Stint,Compound,StintLength
0,ALB,1.0,MEDIUM,12
1,ALB,2.0,HARD,42
2,ALB,3.0,MEDIUM,17
3,ALO,1.0,MEDIUM,31
4,ALO,2.0,HARD,40


In [44]:
stint_with_pos=stints.merge(final_pos, left_on=["Driver"],right_on=["Driver"]).sort_values(by=['Position','Stint'])
stint_with_pos.head()

,Driver,Stint,Compound,StintLength,Position
19,NOR,1.0,MEDIUM,28,1.0
20,NOR,2.0,HARD,44,1.0
41,VER,1.0,MEDIUM,27,2.0
42,VER,2.0,HARD,45,2.0
15,LEC,1.0,MEDIUM,24,3.0


In [45]:
fig=px.bar(
    stint_with_pos,
    y="Driver",
    x='StintLength',
    text="StintLength",
    orientation='h',
    barmode='stack',
    color='Compound',
    template="simple_white",
    title="<b>Tyre Strategies for the {} {}</b>".format(year,gp),
    height=800, width=1200,
    hover_data=["Position"],
    color_discrete_map={
                 "SOFT": "#F91536",
                 "MEDIUM": "#ffdf00 ",
                 "HARD":"#c8c8c8 ",
                 "INTERMEDIATE":"#009E60 ",
                 "WET":"#3671C6"
    }
)
fig.update_layout(
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    xaxis_title="<b>Lap Number</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)

fig.update_layout(yaxis={'categoryorder':'total descending'},    
        hoverlabel=dict(
        bgcolor="Black",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    margin=dict(l=5, r=5, t=35, b=5),
    )

fig['layout']['yaxis']['autorange'] = "reversed"

fig.update_traces(marker_line_width=1, marker_line_color="WHITE", opacity=0.99, insidetextanchor="middle",textfont_color="White")

fig.update_yaxes(ticksuffix = "  ")

fig.show()

In [46]:
fig.write_html("{} {} Tyre Strategies.html".format(year,gp),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi Tracks

In [14]:
#2024
# race_multi=["Bahrain Grand Prix", "Saudi Arabian Grand Prix", "Australian Grand Prix", "Japanese Grand Prix","Chinese Grand Prix","Miami Grand Prix","Emilia Romagna Grand Prix","Monaco Grand Prix","Canadian Grand Prix","Spanish Grand Prix","Austrian Grand Prix","British Grand Prix","Hungarian Grand Prix","Belgian Grand Prix"]
#2023
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2020
for i in race_multi:
    track2=track3[(track3["GP"]==i)&(track3["Year"]==year_multi)]
    track_all_laps_2=track_all_laps[(track_all_laps["GP"]==i)&(track_all_laps["Year"]==year_multi)]
    final_pos=track_all_laps_2[track_all_laps_2["LapNumber"]==track_all_laps_2.LapNumber.max()][["Driver","Position"]]
    year=track2['Year'].iloc[0]
    gp=track2['GP'].iloc[0]
    Drivers=(track2['Driver'].unique())
    stints = track2[["Driver", "Stint", "Compound", "LapNumber"]]
    stints = stints.groupby(["Driver", "Stint", "Compound"]).count().reset_index()
    stints = stints.rename(columns={"LapNumber": "StintLength"})
    stints.head()
    stint_with_pos=stints.merge(final_pos, left_on=["Driver"],right_on=["Driver"]).sort_values(by=['Position','Stint'])
    stint_with_pos.head()
    fig=px.bar(
        stint_with_pos,
        y="Driver",
        x='StintLength',
        text="StintLength",
        orientation='h',
        barmode='stack',
        color='Compound',
        template="simple_white",
        title="<b>Tyre Strategies for the {} {}</b>".format(year_multi,i),
        height=800, width=1200,
        hover_data=["Position"],
        color_discrete_map={
                    "SOFT": "#F91536",
                    "MEDIUM": "#ffdf00 ",
                    "HARD":"#c8c8c8 ",
                    "INTERMEDIATE":"#009E60 ",
                    "WET":"#3671C6"
        }
    )
    fig.update_layout(
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        xaxis_title="<b>Lap Number</b>",
        yaxis_title="<b>Driver</b>",
        title_x=0.5,
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )

    fig.update_layout(yaxis={'categoryorder':'total descending'},    
            hoverlabel=dict(
            bgcolor="Black",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        margin=dict(l=5, r=5, t=35, b=5),
        )

    fig['layout']['yaxis']['autorange'] = "reversed"

    fig.update_traces(marker_line_width=1, marker_line_color="WHITE", opacity=0.99, insidetextanchor="middle",textfont_color="White")

    fig.update_yaxes(ticksuffix = "  ")
    fig.write_html("{} {} Tyre Strategies.html".format(year_multi,i),full_html=False, include_plotlyjs='cdn')
